# Deploymnet

In [40]:
import aiohttp
import asyncio
import uvicorn
from fastai import *
from fastai.vision import *
from io import BytesIO
from starlette.applications import Starlette
from starlette.middleware.cors import CORSMiddleware
from starlette.responses import HTMLResponse, JSONResponse
from starlette.staticfiles import StaticFiles

In [41]:
export_file_url = 'https://drive.google.com/uc?id=1JyjIpXsvWTWY2mvJz5LDNJ6UCgFp5-ME'
export_file_name = 'bbc-classifier.pkl'
path = Path('/tmp')

In [42]:
import gdown

# url = 'https://drive.google.com/uc?id=0B9P1L--7Wd2vNm9zMTJWOGxobkU'
# output = '20150428_collected_images.tgz'
gdown.download(export_file_url, str(path/export_file_name), quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1JyjIpXsvWTWY2mvJz5LDNJ6UCgFp5-ME
To: /tmp/bbc-classifier.pkl

0.00B [00:00, ?B/s]
524kB [00:00, 3.55MB/s]
4.19MB [00:00, 4.81MB/s]
9.44MB [00:00, 6.60MB/s]
14.7MB [00:00, 8.68MB/s]
19.9MB [00:00, 11.3MB/s]
23.1MB [00:00, 12.4MB/s]
27.8MB [00:01, 15.5MB/s]
33.6MB [00:01, 17.5MB/s]
40.9MB [00:01, 22.5MB/s]
45.1MB [00:01, 19.1MB/s]
49.8MB [00:01, 22.4MB/s]
55.6MB [00:01, 25.8MB/s]
59.2MB [00:02, 25.9MB/s]
62.9MB [00:02, 23.6MB/s]
66.1MB [00:02, 24.2MB/s]
69.2MB [00:02, 19.2MB/s]
71.8MB [00:02, 15.7MB/s]
76.5MB [00:03, 18.2MB/s]
79.2MB [00:03, 18.7MB/s]
81.8MB [00:03, 15.2MB/s]
84.4MB [00:03, 16.5MB/s]
87.4MB [00:03, 23.7MB/s]


'/tmp/bbc-classifier.pkl'

In [22]:
import wget

In [24]:
wget.download(export_file_url, str(path/export_file_name))

'/tmp/bbc-classifier (1).pkl'

In [25]:
async def download_file(url, dest):
    if dest.exists(): return
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            data = await response.read()
            with open(dest, 'wb') as f:
                f.write(data)

In [12]:
async def setup_learner():
    await download_file(export_file_url, path / export_file_name)
    try:
        learn = load_learner(path, export_file_name)
        return learn
    except RuntimeError as e:
        if len(e.args) > 0 and 'CPU-only machine' in e.args[0]:
            print(e)
            message = "\n\nThis model was trained with an old version of fastai and will not work in a CPU environment.\n\nPlease update the fastai library in your training environment and export your model again.\n\nSee instructions for 'Returning to work' at https://course.fast.ai."
            raise RuntimeError(message)
        else:
            raise


In [26]:
path = Path('/tmp')

In [28]:
await download_file(export_file_url, path / export_file_name)

In [18]:
export_file_name

'bbc-classifier.pkl'

In [20]:
learn = load_learner(Path('~/Downloads').expanduser(), export_file_name)

/home/stephen/anaconda3/envs/dsgo/lib/python3.7/site-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'torch.nn.modules.loss.CrossEntropyLoss' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/stephen/anaconda3/envs/dsgo/lib/python3.7/site-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/stephen/anaconda3/envs/dsgo/lib/python3.7/site-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve 

In [16]:
learn=setup_learner()

In [ ]:
learn.

In [13]:
loop = asyncio.get_event_loop()
tasks = [asyncio.ensure_future(setup_learner())]
learn = loop.run_until_complete(asyncio.gather(*tasks))[0]
loop.close()

RuntimeError: This event loop is already running